In [ ]:
%%sql -r dataframe_1
/***********************************************
アカウント全体の予算を作成し、
月全体のすべてのコンピューティング リソースを監視し、
支出を設定された制限と比較
************************************************/

In [ ]:
%%sql -r dataframe_3
--ロールをACCOUNTADMINに設定
USE ROLE ACCOUNTADMIN;
--作業全体で使用する名前を定義
SET my_email = 'iyama.kayo.dk@gmail.com'; -- 自分の認証済みメール
SET wh_name  = 'FF_WH';
SET db_name  = 'TRY_DB';
SET sc_name  = 'WEEK89_SCHEMA';
SET custom_budget_name = 'FF_BUDGET_WEEK89';

In [ ]:
%%sql -r dataframe_2
--Snowflakeからメールを送るための「公式な通信経路」を確立
CREATE OR REPLACE NOTIFICATION INTEGRATION budgets_notification_integration
    TYPE = EMAIL
    ENABLED = TRUE
    ALLOWED_RECIPIENTS = ($my_email);

-- システムへの使用許可
GRANT USAGE 
    ON INTEGRATION budgets_notification_integration 
    TO APPLICATION snowflake;

In [ ]:
%%sql -r dataframe_17
-- データベース作成
CREATE OR REPLACE DATABASE IDENTIFIER($db_name);


In [ ]:
%%sql -r dataframe_18
-- スキーマ作成
--直前にDBを作成すると、Snowflakeは自動的にそのDBをカレント(USE)にセットしてくれる
CREATE OR REPLACE SCHEMA IDENTIFIER($sc_name);


In [ ]:
%%sql -r dataframe_19
--アカウント予算管理用ロールの作成
CREATE OR REPLACE ROLE account_budget_admin;

-- 権限の付与（最強鍵とシステム閲覧権）
GRANT APPLICATION ROLE SNOWFLAKE.BUDGET_ADMIN 
    TO ROLE account_budget_admin;

GRANT IMPORTED PRIVILEGES 
    ON DATABASE SNOWFLAKE 
    TO ROLE account_budget_admin;




In [ ]:
%%sql -r dataframe_4
--アカウント予算閲覧用ロールの作成
CREATE OR REPLACE ROLE account_budget_monitor;

-- 「見るだけ」の権限を付与
GRANT APPLICATION ROLE SNOWFLAKE.BUDGET_VIEWER 
    TO ROLE account_budget_monitor;

-- システムデータの閲覧許可
GRANT IMPORTED PRIVILEGES 
    ON DATABASE SNOWFLAKE 
    TO ROLE account_budget_monitor;


In [ ]:
%%sql -r dataframe_5
--ウェアハウスを作成する
CREATE OR REPLACE WAREHOUSE IDENTIFIER($wh_name) 
    WAREHOUSE_SIZE = 'XSMALL'
    AUTO_SUSPEND = 60; --60秒 続いたら、ウェアハウスを自動的に停止（Suspend）させる

In [ ]:
%%sql -r dataframe_8
USE ROLE ACCOUNTADMIN;
-- 1. アカウント予算管理ロールを自分に付与
GRANT ROLE account_budget_admin TO USER IYAMA;
-- 2. アカウント予算監視ロールを自分に付与
GRANT ROLE account_budget_monitor  TO USER IYAMA;

In [ ]:
%%sql -r dataframe_7
-- 作成したロールを一覧表示する
SHOW ROLES LIKE '%BUDGET%';
--チェックポイント:
--account_budget_admin と account_budget_monitor がリストに出てくれば、
--ロールの作成は成功

In [ ]:
%%sql -r dataframe_9
-- 特定のロールにどんな権限（または他のロール）が紐付いているか確認
SHOW GRANTS TO ROLE account_budget_admin;
SHOW GRANTS TO ROLE account_budget_monitor;

--チェックポイント:
--admin の方には SNOWFLAKE.BUDGET_ADMIN が含まれているはずです。
--monitor の方には SNOWFLAKE.BUDGET_VIEWER が含まれているはずです。

In [ ]:
%%sql -r dataframe_6
USE ROLE ACCOUNTADMIN;

-- アカウント全体の予算機能をオンにする
CALL SNOWFLAKE.LOCAL.ACCOUNT_ROOT_BUDGET!ACTIVATE();

-- 1ヶ月の支出上限を100クレジットに設定
CALL SNOWFLAKE.LOCAL.ACCOUNT_ROOT_BUDGET!SET_SPENDING_LIMIT(100);

-- 通知を受け取るメールと通信経路を指定
CALL SNOWFLAKE.LOCAL.ACCOUNT_ROOT_BUDGET!SET_EMAIL_NOTIFICATIONS(
    'budgets_notification_integration',
    $my_email
);

In [ ]:
%%sql -r dataframe_20
USE ROLE account_budget_monitor;

CALL snowflake.local.account_root_budget!GET_CONFIG();

In [ ]:
%%sql -r dataframe_10
USE ROLE account_budget_monitor;

-- ① まず履歴を取得（結果セットを作る）
CALL snowflake.local.account_root_budget!GET_SPENDING_HISTORY(
  TIME_LOWER_BOUND => DATE_TRUNC('MONTH', CURRENT_TIMESTAMP()),
  TIME_UPPER_BOUND => CURRENT_TIMESTAMP()
);

-- ② 直前のCALL結果を集計（当月の累計クレジット）
SELECT
  SUM(CREDITS_SPENT) AS month_to_date_credits
FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()));

In [ ]:
%%sql -r dataframe_21
USE ROLE account_budget_monitor;
--「当月ここまでの平均消費ペース」を使って、月末まで線形外挿
CALL snowflake.local.account_root_budget!GET_SPENDING_HISTORY(
  TIME_LOWER_BOUND => DATE_TRUNC('MONTH', CURRENT_TIMESTAMP()),
  TIME_UPPER_BOUND => CURRENT_TIMESTAMP()
);

WITH daily AS (
  SELECT
    MEASUREMENT_DATE,
    SUM(CREDITS_SPENT) AS credits
  FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()))
  GROUP BY 1
),
mtd AS (
  SELECT
    SUM(credits) AS month_to_date_credits,
    COUNT(*)      AS days_with_usage
  FROM daily
),
cal AS (
  SELECT
    DAY(LAST_DAY(CURRENT_DATE())) AS days_in_month
)
SELECT
  month_to_date_credits                                       AS current_spending,
  (month_to_date_credits / NULLIF(days_with_usage, 0))
    * days_in_month                                           AS predicted_spending_simple,
  days_with_usage,
  days_in_month
FROM mtd, cal;

In [ ]:
%%sql -r dataframe_23
USE ROLE account_budget_monitor;

CALL snowflake.local.account_root_budget!GET_SPENDING_HISTORY(
  TIME_LOWER_BOUND => DATEADD('day', -6, CURRENT_DATE()),  -- 今日含め7日
  TIME_UPPER_BOUND => DATEADD('day',  1, CURRENT_DATE())   -- 明日0:00未満まで
);

SELECT
  SUM(CREDITS_SPENT) AS credits_last_7_days
FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()));

In [ ]:
%%sql -r dataframe_24
USE ROLE account_budget_monitor;

-- 例：2026-02-01〜2026-02-15（終了日は「含めない」運用にすると事故りにくい）
CALL snowflake.local.account_root_budget!GET_SPENDING_HISTORY(
  TIME_LOWER_BOUND => TO_TIMESTAMP_NTZ('2026-02-25 00:00:00'),
  TIME_UPPER_BOUND => TO_TIMESTAMP_NTZ('2026-02-27 00:00:00')
);

SELECT
  SUM(CREDITS_SPENT) AS credits_in_range
FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()));

In [ ]:
%%sql -r dataframe_22
USE ROLE account_budget_monitor;

CALL snowflake.local.account_root_budget!GET_SPENDING_HISTORY(
  TIME_LOWER_BOUND => DATEADD('day', -6, CURRENT_DATE()),
  TIME_UPPER_BOUND => DATEADD('day',  1, CURRENT_DATE())
);

SELECT
  MEASUREMENT_DATE,
  SUM(CREDITS_SPENT) AS credits
FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()))
GROUP BY 1
ORDER BY 1;

In [ ]:
%%sql -r dataframe_11
CALL snowflake.local.account_root_budget!GET_SPENDING_HISTORY(
    TIME_LOWER_BOUND => DATEADD('days', -30, CURRENT_TIMESTAMP()),
    TIME_UPPER_BOUND => CURRENT_TIMESTAMP()
);

In [ ]:
%%sql -r dataframe_12
/***********************************************
アカウント内の特定のリソースセット（Snowflakeオブジェクト）に合わせて予算を作成。
これは、事業部門、部署、またはアプリケーションに対応する場合もある。
その後、その月の割り当てられた制限と比較して、支出額を追跡。
************************************************/

In [ ]:
%%sql -r dataframe_26
USE ROLE ACCOUNTADMIN;

-- 変数から完全修飾スキーマ名を組み立て（IDENTIFIERは1つにまとめる）
SET schema_fqn = $db_name || '.' || $sc_name;

-- 1. 予算の作成者・所有者 (budget_owner)
CREATE OR REPLACE ROLE budget_owner;

GRANT DATABASE ROLE SNOWFLAKE.BUDGET_CREATOR TO ROLE budget_owner;  -- 予算作成に必要 :contentReference[oaicite:2]{index=2}
GRANT CREATE SNOWFLAKE.CORE.BUDGET ON SCHEMA IDENTIFIER($schema_fqn) TO ROLE budget_owner; -- 予算作成に必要 :contentReference[oaicite:3]{index=3}
GRANT USAGE ON DATABASE IDENTIFIER($db_name) TO ROLE budget_owner;
GRANT USAGE ON SCHEMA  IDENTIFIER($schema_fqn) TO ROLE budget_owner;

-- 2. カスタム予算の管理者 (budget_admin)
CREATE OR REPLACE ROLE budget_admin;

GRANT USAGE ON DATABASE IDENTIFIER($db_name) TO ROLE budget_admin;
GRANT USAGE ON SCHEMA  IDENTIFIER($schema_fqn) TO ROLE budget_admin;
GRANT DATABASE ROLE SNOWFLAKE.USAGE_VIEWER TO ROLE budget_admin;    -- 監視/管理に必要 :contentReference[oaicite:4]{index=4}

-- 3. カスタム予算の監視者 (budget_monitor)
CREATE OR REPLACE ROLE budget_monitor;

GRANT USAGE ON DATABASE IDENTIFIER($db_name) TO ROLE budget_monitor;
GRANT USAGE ON SCHEMA  IDENTIFIER($schema_fqn) TO ROLE budget_monitor;
GRANT DATABASE ROLE SNOWFLAKE.USAGE_VIEWER TO ROLE budget_monitor;  -- 監視に必要 :contentReference[oaicite:5]{index=5}

-- 全員にウェアハウスの使用権限を付与（必要なら）
GRANT USAGE ON WAREHOUSE IDENTIFIER($wh_name) TO ROLE budget_owner;
GRANT USAGE ON WAREHOUSE IDENTIFIER($wh_name) TO ROLE budget_admin;
GRANT USAGE ON WAREHOUSE IDENTIFIER($wh_name) TO ROLE budget_monitor;

In [ ]:
%%sql -r dataframe_25
USE ROLE ACCOUNTADMIN;
-- 1. カスタム予算作成・所有ロールを自分に付与
GRANT ROLE budget_owner TO USER IYAMA;
-- 2. カスタム予算管理ロールを自分に付与
GRANT ROLE budget_admin TO USER IYAMA;
-- 3. カスタム予算監視ロールを自分に付与
GRANT ROLE budget_monitor TO USER IYAMA;

In [ ]:
%%sql -r dataframe_13
-- 作成者ロール(budget_owner)に切り替え
USE ROLE budget_owner;
USE WAREHOUSE IDENTIFIER($wh_name);
USE DATABASE  IDENTIFIER($db_name);
USE SCHEMA    IDENTIFIER($sc_name);

-- FQNを作る（db.schema.budget）
SET budget_fqn = $db_name || '.' || $sc_name || '.' || $custom_budget_name;

-- 1) 予算オブジェクトの作成（括弧 () はCREATE側だけ）
SET stmt = 'CREATE OR REPLACE SNOWFLAKE.CORE.BUDGET ' || $budget_fqn || '()';
EXECUTE IMMEDIATE $stmt;

-- 2) 上限（10クレジット/月）
SET stmt = 'CALL ' || $budget_fqn || '!SET_SPENDING_LIMIT(10)';
EXECUTE IMMEDIATE $stmt;  -- 例：CALL my_database.my_schema.my_budget!SET_SPENDING_LIMIT(100); :contentReference[oaicite:2]{index=2}

-- 3) 通知設定（email は文字列として渡す）
SET stmt =
  'CALL ' || $budget_fqn || '!SET_EMAIL_NOTIFICATIONS(' ||
  '''budgets_notification_integration'',' ||
  '''' || $my_email || '''' ||
  ')';
EXECUTE IMMEDIATE $stmt;

-- 4) 監視対象（リソース）の紐付け
-- ADD_RESOURCE は「SELECT SYSTEM$REFERENCE(...)」を渡すのが公式例 :contentReference[oaicite:3]{index=3}
SET stmt =
  'CALL ' || $budget_fqn || '!ADD_RESOURCE(' ||
  '  SELECT SYSTEM$REFERENCE(''WAREHOUSE'', ''' || $wh_name || ''', ''SESSION'', ''APPLYBUDGET'')' ||
  ')';
EXECUTE IMMEDIATE $stmt;

SET stmt =
  'CALL ' || $budget_fqn || '!ADD_RESOURCE(' ||
  '  SELECT SYSTEM$REFERENCE(''DATABASE'', ''' || $db_name || ''', ''SESSION'', ''APPLYBUDGET'')' ||
  ')';
EXECUTE IMMEDIATE $stmt;

-- 5) 他ロールに「予算を見る権限」を付与（ADMIN / VIEWER）
-- 作成者(Owner)が、管理者や監視者に「見てもいいよ」と許可を出す
SET stmt =
  'GRANT SNOWFLAKE.CORE.BUDGET ROLE ' || $budget_fqn || '!ADMIN TO ROLE budget_admin';
EXECUTE IMMEDIATE $stmt;

SET stmt =
  'GRANT SNOWFLAKE.CORE.BUDGET ROLE ' || $budget_fqn || '!VIEWER TO ROLE budget_monitor';
EXECUTE IMMEDIATE $stmt;



In [ ]:
%%sql -r dataframe_37
-- 監視ロールに切り替え
USE ROLE budget_monitor;

-- 予算FQN
SET budget_fqn = $db_name || '.' || $sc_name || '.' || $custom_budget_name;

-- 直近7日の支出履歴（Budgets）
SET stmt =
  'CALL ' || $budget_fqn || '!GET_SPENDING_HISTORY(' ||
  '  TIME_LOWER_BOUND => DATEADD(''days'', -7, CURRENT_TIMESTAMP()),' ||
  '  TIME_UPPER_BOUND => CURRENT_TIMESTAMP()' ||
  ')';
EXECUTE IMMEDIATE $stmt;

In [ ]:
%%sql -r dataframe_36
--
USE WAREHOUSE FF_WH;

SELECT COUNT(*)
FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY;

In [ ]:
%%sql -r dataframe_16
-- 1. 最強権限（ACCOUNTADMIN）に戻る
USE ROLE ACCOUNTADMIN;

-- 2. 変数の再確認（削除対象を指定）
SET wh_name  = 'FF_WH';
SET db_name  = 'TRY_DB';
SET sc_name  = 'WEEK89_SCHEMA';
SET custom_budget_name = 'FF_BUDGET_WEEK89';


-- 3. カスタム予算オブジェクトの削除
-- スキーマ内にある予算を先に削除します
SET budget_fqn = $db_name || '.' || $sc_name || '.' || $custom_budget_name;
DROP SNOWFLAKE.CORE.BUDGET IF EXISTS IDENTIFIER($budget_fqn);

-- 4. データベースとウェアハウスの削除（必要に応じて）
--DROP DATABASE IF EXISTS IDENTIFIER($db_name);
--DROP WAREHOUSE IF EXISTS IDENTIFIER($wh_name);

-- 5. 通知インテグレーションの削除
DROP NOTIFICATION INTEGRATION IF EXISTS budgets_notification_integration;

-- 6. 作成したカスタムロールの一括削除
DROP ROLE IF EXISTS account_budget_admin;
DROP ROLE IF EXISTS account_budget_monitor;
DROP ROLE IF EXISTS budget_owner;
DROP ROLE IF EXISTS budget_admin;
DROP ROLE IF EXISTS budget_monitor;

-- 7. アカウント予算の非アクティブ化（必要な場合のみ）
-- アカウント全体の監視機能をオフにします
CALL SNOWFLAKE.LOCAL.ACCOUNT_ROOT_BUDGET!DEACTIVATE();

